<a href="https://colab.research.google.com/github/bartek717/bias_analysis/blob/main/BABEV3_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers
!pip install torch
# Let's import the library and load the dataset:
!pip install transformers[torch]
!pip install accelerate -U
!pip install torch torchvision
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, Dataset
import ast
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification, DistilBertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import ast
from sklearn.metrics import classification_report
from google.colab import drive
drive.mount('/content/drive')
save_directory = "/content/drive/My Drive/results"
save_directory2 = "/content/drive/My Drive/results2"

import os
os.makedirs(save_directory, exist_ok=True)
os.makedirs(save_directory2, exist_ok=True)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 9.5 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
dataset = load_dataset("mediabiasgroup/BABE-v3")

train_val_test = dataset['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = train_val_test['train']
test_val_dataset = train_val_test['test']
test_val = test_val_dataset.train_test_split(test_size=0.5, seed=42)
test_dataset = test_val['train']
val_dataset = test_val['test']

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
model = DistilBertForTokenClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokens = tokenizer.tokenize("Yes, the rich are rich and want to stay that way, and yet at the same time, fat cats now lean left, even far left, on cultural issues–and they want you to do as you’re told.")
print(tokens)


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['yes', ',', 'the', 'rich', 'are', 'rich', 'and', 'want', 'to', 'stay', 'that', 'way', ',', 'and', 'yet', 'at', 'the', 'same', 'time', ',', 'fat', 'cats', 'now', 'lean', 'left', ',', 'even', 'far', 'left', ',', 'on', 'cultural', 'issues', '–', 'and', 'they', 'want', 'you', 'to', 'do', 'as', 'you', '’', 're', 'told', '.']


In [ ]:

def process_data_to_labels(sentences, biased_words_list):
    labels = []
    tokensHead = []
    for sentence, biased_words in zip(sentences, biased_words_list):
        if biased_words == "nan":
            biased_words = []
        else:
            try:
                biased_words = ast.literal_eval(biased_words)
                if not isinstance(biased_words, list):
                    biased_words = []
            except (SyntaxError, ValueError):
                biased_words = []

        tokens = tokenizer.tokenize(sentence)
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        tokensHead.append(tokens)
        label = [0] * len(tokens)
        for i, token in enumerate(tokens):
            for word in biased_words:
                if word == token:
                    label[i] = 1

        labels.append(label)
    train_labels = labels
    return train_labels, tokensHead

train_labels, tokensHead = process_data_to_labels(dataset['train']['text'], dataset['train']['biased_words'])

In [ ]:
for i in range(4):
  print("First Passage:", dataset['train']['text'][i])
  print("Biased Words:", dataset['train']['biased_words'][i])
  print("Labels:", train_labels[i])

First Passage: But instead of closing loopholes that allow criminals to access firearms, the bill by House Democrats could turn millions of law-abiding American gun owners into felons.
Biased Words: []
Labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
First Passage: "You think I'm joking," he continued. "
Biased Words: nan
Labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
First Passage: Yes, the rich are rich and want to stay that way, and yet at the same time, fat cats now lean left, even far left, on cultural issues–and they want you to do as you’re told. 
Biased Words: ['fat', 'cats']
Labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
First Passage: Americans are carrying $1.57 trillion in outstanding student loan debt, more than what they owe on their credit cards.
Biased Words: []
Labels: [0, 0, 0, 0, 0, 0, 

In [ ]:
train_encodings = tokenizer(dataset['train']['text'], truncation=True, padding=True, is_split_into_words=False, return_tensors="pt")
train_labels_padded = []
for label in train_labels:
    print(label)
    print(train_encodings['input_ids'].size(1))
    train_labels_padded.append(label + [0] * (train_encodings['input_ids'].size(1) - len(label)))

print(train_labels_padded[0])
print(train_labels_padded[1])
print(train_labels_padded[2])
print(train_labels_padded[3])
print(train_labels_padded[4])
train_labels_tensor = torch.tensor(train_labels_padded)

Streaming output truncated to the last 5000 lines.
119
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
119
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
119
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
119
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
119
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
119
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
119
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
BATCH_SIZE = 16
LEARNING_RATE = 5e-5
EPOCHS = 5
GRADIENT_CLIP = 1.0
WARMUP_STEPS = int(0.1 * len(train_encodings['input_ids']) / BATCH_SIZE)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=len(train_encodings['input_ids']) * EPOCHS / BATCH_SIZE)

model.train()

for epoch in range(EPOCHS):
    total_loss = 0.0

    for i in range(0, len(train_encodings['input_ids']), BATCH_SIZE):
        optimizer.zero_grad()

        batch_input_ids = train_encodings['input_ids'][i:i+BATCH_SIZE].to(device)
        batch_attention_mask = train_encodings['attention_mask'][i:i+BATCH_SIZE].to(device)
        batch_labels = train_labels_tensor[i:i+BATCH_SIZE].to(device)

        outputs = model(batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
        loss = outputs.loss
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIP)

        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    print(f"Epoch: {epoch}, Loss: {total_loss / len(train_encodings['input_ids'])}")



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 0, Loss: 0.003038686812145455


KeyboardInterrupt: ignored

In [ ]:
test_encodings = tokenizer(test_dataset['text'], truncation=True, padding=True, is_split_into_words=False, return_tensors="pt")

test_labels, _ = process_data_to_labels(test_dataset['text'], test_dataset['biased_words'])
test_labels_padded = []
for label in test_labels:
    test_labels_padded.append(label + [0] * (test_encodings['input_ids'].size(1) - len(label)))

test_labels_tensor = torch.tensor(test_labels_padded)

model.eval()
predictions = []
true_labels = []

In [ ]:
with torch.no_grad():
    for i in range(len(test_encodings['input_ids'])):
        input_ids = test_encodings['input_ids'][i].to(device).unsqueeze(0)
        attention_mask = test_encodings['attention_mask'][i].to(device).unsqueeze(0)
        labels_i = test_labels_tensor[i].to(device).unsqueeze(0)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        predictions.extend(torch.argmax(logits, dim=-1).flatten().cpu().tolist())
        true_labels.extend(labels_i.flatten().cpu().tolist())

# 3. Calculate the metrics

print(classification_report(true_labels, predictions, target_names=["unbiased", "biased"]))

              precision    recall  f1-score   support

    unbiased       1.00      1.00      1.00     43766
      biased       0.97      0.98      0.98       318

    accuracy                           1.00     44084
   macro avg       0.98      0.99      0.99     44084
weighted avg       1.00      1.00      1.00     44084



In [ ]:

def predict_bias(text, model, tokenizer):
    # Step 1: Tokenize the input text
    inputs = tokenizer.encode_plus(text, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Step 2: Run the model on the tokenized input
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    # Step 3: Interpret the predictions
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    bias_predictions = ["BIASED" if label == 1 else "unbiased" for label in predictions[0]]

    return tokens, predictions[0]

# Test the function with a custom piece of text
from transformers import BertTokenizer, BertForMaskedLM


mlm_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
mlm_model = BertForMaskedLM.from_pretrained('bert-base-uncased').to(device)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# text = ""
# text, bias = predict_bias(text, model, tokenizer)
# print(text)
# print(bias)

In [ ]:
# Given tokens and labels, mask the words with label 1
def mask_tokens(tokens, labels):
    masked_tokens = []
    mask_indices = []
    for i, (token, label) in enumerate(zip(tokens, labels)):
        if label == 1:
            masked_tokens.append('[MASK]')
            mask_indices.append(i)
        else:
            masked_tokens.append(token)
    return masked_tokens, mask_indices


In [ ]:

# masked_tokens, mask_indices = mask_tokens(text, bias)
# print("Masked Tokens:", masked_tokens)



In [ ]:
# # Predict using BertForMaskedLM
# inputs = mlm_tokenizer.encode_plus(' '.join(masked_tokens), return_tensors="pt", truncation=True, padding=True)
# input_ids = inputs["input_ids"].to(device)
# attention_mask = inputs["attention_mask"].to(device)

# with torch.no_grad():
#     outputs = mlm_model(input_ids, attention_mask=attention_mask)
# logits = outputs.logits

# predicted_indices = torch.argmax(logits, dim=2)
# print(predicted_indices)
# mask_indices = [idx + 1 for idx in mask_indices]

In [ ]:
# predicted_tokens2 = mlm_tokenizer.convert_ids_to_tokens(predicted_indices[0].tolist())
# predicted_words2 = [predicted_tokens2[i] for i in mask_indices]
# print("Predicted Words for [MASK]s:", predicted_words2)

In [ ]:
def replace_biased_words(text, bias_model, bias_tokenizer, replace_model, replace_tokenizer):
    # Predict the biased words
    # print(text)
    tokens, bias = predict_bias(text, bias_model, bias_tokenizer)
    masked_tokens, mask_indices = mask_tokens(tokens, bias)
    inputs = mlm_tokenizer.encode_plus(' '.join(masked_tokens), return_tensors="pt", truncation=True, padding=True)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    # print(masked_tokens)
    with torch.no_grad():
        outputs = mlm_model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    predicted_indices = torch.argmax(logits, dim=2)
    mask_indices = [idx + 1 for idx in mask_indices]
    predicted_tokens = mlm_tokenizer.convert_ids_to_tokens(predicted_indices[0].tolist())
    predicted_words = [predicted_tokens[i] for i in mask_indices]
    # Convert tokens back to a single string
    unmasked_text = ' '.join(masked_tokens).replace('[CLS]', '').replace('[SEP]', '').replace('##', '')
    for i in range(len(predicted_words)):
      masked_tokens[mask_indices[i]-1] = predicted_words[i]

    input_array = [x for x in masked_tokens[1:-1]]

    punc_b = [',', '.', '?', '!', ')']
    punc_ba = ["'", "\"", "’", "-"]
    punc_a = ['(']
    quote = ['"']


    output_string = ""
    firstquote = True
    # print(input_array)
    for word in input_array:
        if word.startswith("##"):
            if len(output_string) > 0:
              output_string = output_string[:-1]
            output_string += word[2:] + ' '
        elif word in quote:
          if firstquote:
            output_string += word
            firstquote=False
          else:
            firsquote = True
            output_string = output_string[:-1]
            output_string += word + ' '
        elif word in punc_b:
          output_string = output_string[:-1]
          output_string += word + ' '
        elif word in punc_ba:
          output_string = output_string[:-1]
          output_string += word
        elif word in punc_a:
          output_string += word + ' '
        else:
            output_string += word + ' '


    # Remove the trailing space
    output_string = output_string.strip()
    output_string = output_string.capitalize()
    return output_string


new_dataset_texts = []

for text in dataset['train']['text']:
    new_text = replace_biased_words(text, model, tokenizer, mlm_model, mlm_tokenizer)
    new_dataset_texts.append(new_text)


In [ ]:
print(new_dataset_texts)

['But instead of closing loopholes that allow criminals to access firearms, the bill by house democrats could turn millions of law-abiding american gun owners into felons.', '"you think i\'m joking," he continued."', 'Yes, the rich are rich and want to stay that way, and yet at the same time, they they now lean left, even far left, on cultural issues – and they want you to do as you’re told.', 'Americans are carrying $ 1. 57 trillion in outstanding student loan debt, more than what they owe on their credit cards.', 'Republican state legislators from texas to arizona, to wisconsin, where covid-19 cases are now shooting through the roof, all rode the “ reopening ” bandwagon for months while the band # on # facebook pages aboutn “ patriotism, ” attending rallies in support of gun-toting neo-nazi militias, and bringing frivolous lawsuits to force businesses to reopen.', 'Meanwhile, a pervasive celebrity culture — in which celebrities are considered trusted “ experts ” solely because of the

In [ ]:

from transformers import DataCollatorForLanguageModeling

In [ ]:
model_name_or_path = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)



In [ ]:
print(train_dataset)
print(val_dataset)

# Setting the padding token
tokenizer.pad_token = tokenizer.eos_token or '[PAD]'

# Convert your data into the format expected by GPT-2
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=100)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# Removing unnecessary columns
columns_to_remove = ['text', 'news_link', 'outlet', 'topic', 'type', 'label', 'label_opinion', 'biased_words']
tokenized_train_dataset = tokenized_train_dataset.remove_columns(columns_to_remove)
tokenized_val_dataset = tokenized_val_dataset.remove_columns(columns_to_remove)

print(tokenized_train_dataset)
print(tokenized_val_dataset)

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=7,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_dir='./logs',
    logging_steps=100,
    num_train_epochs=1,
    save_total_limit=2,
    output_dir=save_directory
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We are not doing masked language modeling
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()



Dataset({
    features: ['text', 'news_link', 'outlet', 'topic', 'type', 'label', 'label_opinion', 'biased_words'],
    num_rows: 3296
})
Dataset({
    features: ['text', 'news_link', 'outlet', 'topic', 'type', 'label', 'label_opinion', 'biased_words'],
    num_rows: 413
})


Map:   0%|          | 0/3296 [00:00<?, ? examples/s]

Map:   0%|          | 0/413 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3296
})
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 413
})


Step,Training Loss,Validation Loss
500,3.242600,3.155146


TrainOutput(global_step=824, training_loss=3.28192753467745, metrics={'train_runtime': 448.8239, 'train_samples_per_second': 7.344, 'train_steps_per_second': 1.836, 'total_flos': 597851072102400.0, 'train_loss': 3.28192753467745, 'epoch': 1.0})

In [ ]:
results = trainer.evaluate()
print(results)
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)
model.save_pretrained(save_directory+"model")

{'eval_loss': 3.1252222061157227, 'eval_runtime': 9.755, 'eval_samples_per_second': 42.337, 'eval_steps_per_second': 6.048, 'epoch': 1.0}


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained(save_directory)
model = GPT2LMHeadModel.from_pretrained(save_directory)


In [ ]:
# Use encode to convert input text to tensor
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate text from the model's predictions.
output = model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.7)

# Use decode to convert tensor back to text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the United States was a country of immigrants. Today, it is a country of immigrants.   The American dream is a dream that has been destroyed by the Trump administration.  The American dream is a dream that has been destroyed by the Trump administration. The American dream is a dream that has been destroyed by the Trump administration. The American dream is a dream that has been destroyed by


In [ ]:
new_dataset_texts.pop()
data_dict = {'text': new_dataset_texts}

# Convert the dictionary to a Hugging Face Dataset
df = Dataset.from_dict(data_dict)
print(df)
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=100)

tokenized_train_dataset = df.map(tokenize_function, batched=True)
# tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

Dataset({
    features: ['text'],
    num_rows: 4120
})


Map:   0%|          | 0/4120 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_train_dataset)

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 4120
})


In [ ]:

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=7,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_dir='./logs2',
    logging_steps=100,
    num_train_epochs=1,
    save_total_limit=2,
    output_dir=save_directory2
    )

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We are not doing masked language modeling
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()


Step,Training Loss,Validation Loss
500,3.139100,3.237097
1000,3.133800,3.167632


TrainOutput(global_step=1030, training_loss=3.1659321386837265, metrics={'train_runtime': 619.738, 'train_samples_per_second': 6.648, 'train_steps_per_second': 1.662, 'total_flos': 747313840128000.0, 'train_loss': 3.1659321386837265, 'epoch': 1.0})

In [ ]:
results = trainer.evaluate()
print(results)
tokenizer.save_pretrained(save_directory2)
model.save_pretrained(save_directory2)

{'eval_loss': 3.1677908897399902, 'eval_runtime': 9.9069, 'eval_samples_per_second': 41.688, 'eval_steps_per_second': 5.955, 'epoch': 1.0}


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(save_directory2)
model = GPT2LMHeadModel.from_pretrained(save_directory2)

input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate text from the model's predictions.
output = model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.7)

# Use decode to convert tensor back to text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the left was a force to be reckoned with. it was the party of the left, the party of the left, the party of the left. it was the party of the left, the party of the left. it was the party of the left, the party of the left. it was the party of the left, the party of the left. it was the party of the left, the party of the left. it was the party of the left, the


In [ ]:
# Install the SDK
!curl https://sdk.cloud.google.com | bash
!gcloud init
!gcloud auth login

Streaming output truncated to the last 5000 lines.
google-cloud-sdk/lib/surface/services/quota/list.py
google-cloud-sdk/lib/surface/services/quota/update.py
google-cloud-sdk/lib/surface/services/vpc_peerings/__init__.py
google-cloud-sdk/lib/surface/services/vpc_peerings/connect.py
google-cloud-sdk/lib/surface/services/vpc_peerings/delete.py
google-cloud-sdk/lib/surface/services/vpc_peerings/disable_vpc_service_controls.py
google-cloud-sdk/lib/surface/services/vpc_peerings/enable_vpc_service_controls.py
google-cloud-sdk/lib/surface/services/vpc_peerings/list.py
google-cloud-sdk/lib/surface/services/vpc_peerings/operations/__init__.py
google-cloud-sdk/lib/surface/services/vpc_peerings/operations/describe.py
google-cloud-sdk/lib/surface/services/vpc_peerings/operations/wait.py
google-cloud-sdk/lib/surface/services/vpc_peerings/update.py
google-cloud-sdk/lib/surface/source/__init__.py
google-cloud-sdk/lib/surface/source/project_configs/__init__.py
google-cloud-sdk/lib/surface/source/projec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# !gsutil cp -r "/content/drive/My Drive/results2" gs://bias_bucket/
!gsutil cp -r "/content/drive/My Drive/resultsmodel" gs://bias_bucket/




Mounted at /content/drive
Copying file:///content/drive/My Drive/resultsmodel/generation_config.json [Content-Type=application/json]...
Copying file:///content/drive/My Drive/resultsmodel/config.json [Content-Type=application/json]...
Copying file:///content/drive/My Drive/resultsmodel/pytorch_model.bin [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads

In [ ]:
!pip install onnx
!pip install tf2onnx
!curl https://sdk.cloud.google.com | bash
!gcloud init
!gcloud auth login
import torch
import onnx
from tf2onnx import convert


from google.colab import drive
drive.mount('/content/drive')

# !gsutil cp -r "/content/drive/My Drive/resultsmodel" gs://bias_bucket/
# model = GPT2LMHeadModel.from_pretrained(save_directory+'model')


Streaming output truncated to the last 5000 lines.
google-cloud-sdk/lib/surface/spanner/databases/list.py
google-cloud-sdk/lib/surface/spanner/databases/remove_iam_policy_binding.yaml
google-cloud-sdk/lib/surface/spanner/databases/restore.py
google-cloud-sdk/lib/surface/spanner/databases/roles/__init__.py
google-cloud-sdk/lib/surface/spanner/databases/roles/list.py
google-cloud-sdk/lib/surface/spanner/databases/sessions/__init__.py
google-cloud-sdk/lib/surface/spanner/databases/sessions/delete.py
google-cloud-sdk/lib/surface/spanner/databases/sessions/list.py
google-cloud-sdk/lib/surface/spanner/databases/set_iam_policy.py
google-cloud-sdk/lib/surface/spanner/databases/update.py
google-cloud-sdk/lib/surface/spanner/instance_configs/__init__.py
google-cloud-sdk/lib/surface/spanner/instance_configs/create.py
google-cloud-sdk/lib/surface/spanner/instance_configs/delete.py
google-cloud-sdk/lib/surface/spanner/instance_configs/describe.py
google-cloud-sdk/lib/surface/spanner/instance_config

In [ ]:


# Load your GPT-2 model from the pytorch_model.bin file
model_path = '/content/drive/My Drive/resultsmodel/pytorch_model.bin'  # Adjust the path
pytorch_model = torch.load(model_path)

# Create a dummy input prompt
dummy_input = "Generate a text sample."

# Export the PyTorch model to ONNX format
onnx_model = convert.from_pytorch(pytorch_model, dummy_input)

# Save the ONNX model
onnx.save(onnx_model, '/content/drive/My Drive/resultsmodel/gpt2_medium.onnx')

AttributeError: ignored